In [1]:
sample_input_1 = """
AAAA
BBCD
BBCC
EEEC
"""

In [2]:
sample_input_2 = """
OOOOO
OXOXO
OOOOO
OXOXO
OOOOO
"""

In [3]:
sample_input_3 = """
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
"""

In [4]:
with open("input.txt") as f:
    real_input =(f.read())

In [5]:
def read_garden(input: str) -> list[list[str]]:
    return [list(row) for row in input.strip().split('\n')]

In [6]:
def fence_amount(position: tuple[int, int], garden: list[list[str]]) -> int:
    height = len(garden)
    width = len(garden[0])
    amount = 0

    if position[1] == 0 or garden[position[1]][position[0]] != garden[position[1]-1][position[0]]:
        amount += 1
    if position[0] == width-1 or garden[position[1]][position[0]] != garden[position[1]][position[0]+1]:
        amount += 1
    if position[1] == height-1 or garden[position[1]][position[0]] != garden[position[1]+1][position[0]]:
        amount += 1
    if position[0] == 0 or garden[position[1]][position[0]] != garden[position[1]][position[0]-1]:
        amount += 1
    
    return amount

In [7]:
def rename_plot(plots: list[list[int]], previous: int, new: int):
    height = len(plots)
    width = len(plots[0])

    for y in range(height):
        for x in range(width):
            if plots[y][x] == previous:
                plots[y][x] = new

def highest_plot(plots: list[list[int]]) -> int:
    return max([max(row) for row in plots])

def find_plots(garden: list[list[str]]):
    height = len(garden)
    width = len(garden[0])

    plots = [[-1] * width for _ in range(height)]

    for y, row in enumerate(garden):
        for x, current_plant in enumerate(row):
            up_plot = None if y == 0 else plots[y-1][x]
            up_plant = None if y == 0 else garden[y-1][x]
            left_plot = None if x == 0 else plots[y][x-1]
            left_plant = None if x == 0 else garden[y][x-1]

            if current_plant == up_plant and current_plant == left_plant:
                if up_plot != left_plot:
                    rename_plot(plots, max(up_plot, left_plot), min(up_plot, left_plot))
                    plots[y][x] = min(up_plot, left_plot)
                else:
                    plots[y][x] = up_plot
            elif current_plant == up_plant:
                plots[y][x] = up_plot
            elif current_plant == left_plant:
                plots[y][x] = left_plot
            else:
                plots[y][x] = highest_plot(plots) + 1
    return plots

In [48]:
# Part 1
garden = read_garden(real_input)
plots = find_plots(garden)

plot_dict = {}
for y, row in enumerate(plots):
    for x, current_plot in enumerate(row):
        if current_plot not in plot_dict.keys():
            plot_dict[current_plot] = []
        plot_dict[current_plot].append(fence_amount((x,y), garden))
sum([sum(plot_dict[plot]) * len(plot_dict[plot]) for plot in plot_dict.keys()])

1371306

In [39]:
def count_corners(position: tuple[int,int], plots: list[list[int]]):
    plot = plots[position[1]][position[0]]
    amount = 0
    height = len(plots)
    width = len(plots[0])

    plot_up = plots[position[1]-1][position[0]] if position[1] > 0 else -1
    plot_right = plots[position[1]][position[0]+1] if position[0] < width-1 else -1
    plot_down = plots[position[1]+1][position[0]] if position[1] < height-1 else -1
    plot_left = plots[position[1]][position[0]-1] if position[0] > 0 else -1

    # outer corner
    if plot != plot_up and plot != plot_left: amount += 1 # TL
    if plot != plot_up and plot != plot_right: amount += 1 # TR
    if plot != plot_down and plot != plot_left: amount += 1 # BL
    if plot != plot_down and plot != plot_right: amount += 1 # BR

    # inner corner
    if plot == plot_up and plot == plot_left and plot != plots[position[1]-1][position[0]-1]: amount += 1 # TL
    if plot == plot_up and plot == plot_right and plot != plots[position[1]-1][position[0]+1]: amount += 1 # TR
    if plot == plot_down and plot == plot_left and plot != plots[position[1]+1][position[0]-1]: amount += 1 # BL
    if plot == plot_down and plot == plot_right and plot != plots[position[1]+1][position[0]+1]: amount += 1 # BR

    return amount

In [49]:
# part 2
plot_corners = {}
plot_area = {}

for y, row in enumerate(plots):
    for x, plot in enumerate(row):
        if plot not in plot_area.keys():
            plot_area[plot] = 1
        else:
            plot_area[plot] += 1

        if plot not in plot_corners.keys():
            plot_corners[plot] = count_corners((x,y), plots)
        else:
            plot_corners[plot] += count_corners((x,y), plots)

plot_sides = plot_corners # polygons: amount of corners == amount of sides

sum([plot_corners[plot] * plot_area[plot] for plot in plot_area.keys()])


805880

In [ ]:
# Visualize map with colors

import numpy as np
import cv2
import random

height = len(garden)
width = len(garden[0])

img_garden = np.zeros((height, width, 3))
img_plots = np.zeros((height, width, 3))

plant_color = {}
plot_color = {}
for y in range(height):
    for x in range(width):
        if garden[y][x] not in plant_color.keys():
            plant_color[garden[y][x]] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        img_garden[y,x] = plant_color[garden[y][x]]

        if plots[y][x] not in plot_color.keys():
            plot_color[plots[y][x]] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        img_plots[y,x] = plot_color[plots[y][x]]

cv2.imwrite('garden.png', img_garden)
cv2.imwrite('plots.png', img_plots)